Cell 1 — Import & Đường dẫn

In [7]:
import os, json, random, numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications.xception import Xception, preprocess_input

In [8]:
# ĐƯỜNG DẪN DỮ LIỆU CỦA BẠN (đã có train/val/test và 3 lớp con)
DATA_DIR = r"C:\Users\VACB\OneDrive\Desktop\xception_v2\chest_xray_v2"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR   = os.path.join(DATA_DIR, "val")
TEST_DIR  = os.path.join(DATA_DIR, "test")

# Thư mục xuất artifacts (model, logs, ảnh Grad-CAM)
ARTIFACTS = os.path.join(DATA_DIR, "artifacts"); os.makedirs(ARTIFACTS, exist_ok=True)
CKPT_DIR  = os.path.join(DATA_DIR, "checkpoints"); os.makedirs(CKPT_DIR, exist_ok=True)
CAM_DIR   = os.path.join(ARTIFACTS, "gradcam"); os.makedirs(CAM_DIR, exist_ok=True)

# Cấu hình chung
IMG_SIZE = (224, 224)  # có thể đổi 299,299 nếu muốn (chậm hơn)
BATCH    = 32
SEED     = 1337
AUTOTUNE = tf.data.AUTOTUNE

print("TF:", tf.__version__)

TF: 2.10.1


# Cell 2: Datasets (RGB) + augment + preprocess. Tạo tf.data pipelines

In [9]:
# Cell 2: Datasets (RGB) + augment + preprocess

# Load từ thư mục: label dạng int (0..NUM_CLASSES-1)
train_raw = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, image_size=IMG_SIZE, batch_size=BATCH, color_mode="rgb",
    label_mode="int", seed=SEED, shuffle=True
)
val_raw = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR, image_size=IMG_SIZE, batch_size=BATCH, color_mode="rgb",
    label_mode="int", seed=SEED, shuffle=False
)
test_raw = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, image_size=IMG_SIZE, batch_size=BATCH, color_mode="rgb",
    label_mode="int", seed=SEED, shuffle=False
)

class_names = train_raw.class_names
NUM_CLASSES = len(class_names)
print("Classes:", class_names)

# Lưu class_names để dùng inference sau này
with open(os.path.join(ARTIFACTS, "class_names.json"), "w", encoding="utf-8") as f:
    json.dump(class_names, f, ensure_ascii=False, indent=2)

# Augment nhẹ để tổng quát hóa
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
])

# Preprocess cho Xception (scale theo imagenet)
def apply_preprocess(x, y):
    x = tf.cast(x, tf.float32)
    x = preprocess_input(x)
    return x, y

train_ds = train_raw.map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(apply_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_raw.map(apply_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
test_ds  = test_raw.map(apply_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

Found 5216 files belonging to 3 classes.
Found 47 files belonging to 3 classes.
Found 624 files belonging to 3 classes.
Classes: ['NORMAL', 'PNEUMONIA_bacteria', 'PNEUMONIA_virus']


Cell 3 — Xây model Xception (3 lớp)

In [10]:
# Cell 3: Build model Xception (3-class)

base = Xception(include_top=False, weights="imagenet", input_shape=IMG_SIZE + (3,))
base.trainable = False  # giai đoạn đầu: freeze backbone

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = inputs
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.35)(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(1e-5))(x)
x = layers.Dropout(0.35)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

# Dùng SparseCategoricalCrossentropy (nhãn int); KHÔNG label_smoothing để tương thích version
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=loss, metrics=["accuracy"])
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                           

Callbacks + Train (phase 1)

In [11]:
# Cell 4: Callbacks + Train phase 1 (freeze backbone)

ckpt_path = os.path.join(CKPT_DIR, "xception_pneu_best.h5")
cbs = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, verbose=1),
    tf.keras.callbacks.CSVLogger(os.path.join(ARTIFACTS, "train_log.csv"), append=False)
]

EPOCHS = 50
hist = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cbs)

# Lưu model cuối phase 1
model.save(os.path.join(ARTIFACTS, "xception_pneu_last.h5"))

Epoch 1/50
163/163 [==============================] - ETA: 0s - loss: 0.6727 - accuracy: 0.7136
Epoch 1: val_loss improved from inf to 0.54267, saving model to C:\Users\VACB\OneDrive\Desktop\xception_v2\chest_xray_v2\checkpoints\xception_pneu_best.h5
163/163 [==============================] - 110s 665ms/step - loss: 0.6727 - accuracy: 0.7136 - val_loss: 0.5427 - val_accuracy: 0.7872 - lr: 0.0010
Epoch 2/50
163/163 [==============================] - ETA: 0s - loss: 0.5747 - accuracy: 0.7554
Epoch 2: val_loss improved from 0.54267 to 0.45818, saving model to C:\Users\VACB\OneDrive\Desktop\xception_v2\chest_xray_v2\checkpoints\xception_pneu_best.h5
163/163 [==============================] - 110s 668ms/step - loss: 0.5747 - accuracy: 0.7554 - val_loss: 0.4582 - val_accuracy: 0.8298 - lr: 0.0010
Epoch 3/50
163/163 [==============================] - ETA: 0s - loss: 0.5609 - accuracy: 0.7542
Epoch 3: val_loss did not improve from 0.45818
163/163 [==============================] - 114s 692ms/s

Fine-tune

In [12]:
# Cell 5: Fine-tune một phần backbone (tùy chọn)

base.trainable = True
# Khóa bớt các lớp đầu, mở các lớp cuối (ví dụ ~40 lớp cuối)
for layer in base.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss, metrics=["accuracy"])
hist_ft = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=cbs)

model.save(os.path.join(ARTIFACTS, "xception_pneu_finetuned.h5"))

Epoch 1/20
163/163 [==============================] - ETA: 0s - loss: 0.5214 - accuracy: 0.7753
Epoch 1: val_loss did not improve from 0.45818
163/163 [==============================] - 194s 1s/step - loss: 0.5214 - accuracy: 0.7753 - val_loss: 0.4899 - val_accuracy: 0.8085 - lr: 1.0000e-04
Epoch 2/20
163/163 [==============================] - ETA: 0s - loss: 0.4516 - accuracy: 0.8102
Epoch 2: val_loss did not improve from 0.45818
163/163 [==============================] - 194s 1s/step - loss: 0.4516 - accuracy: 0.8102 - val_loss: 0.4660 - val_accuracy: 0.7872 - lr: 1.0000e-04
Epoch 3/20
163/163 [==============================] - ETA: 0s - loss: 0.4087 - accuracy: 0.8207
Epoch 3: val_loss did not improve from 0.45818
163/163 [==============================] - 193s 1s/step - loss: 0.4087 - accuracy: 0.8207 - val_loss: 0.5036 - val_accuracy: 0.7872 - lr: 1.0000e-04
Epoch 4/20
163/163 [==============================] - ETA: 0s - loss: 0.3908 - accuracy: 0.8280
Epoch 4: val_loss improved f

In [13]:
# Cell 6: Evaluate test set + báo cáo
from sklearn.metrics import confusion_matrix, classification_report

test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test — Loss: {test_loss:.4f} | Acc: {test_acc*100:.2f}%")

y_true, y_pred = [], []
for xb, yb in test_ds:
    probs = model.predict(xb, verbose=0)
    y_true.extend(yb.numpy().tolist())
    y_pred.extend(np.argmax(probs, axis=1).tolist())

print(classification_report(y_true, y_pred, target_names=class_names))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))

Test — Loss: 0.6198 | Acc: 73.72%
                    precision    recall  f1-score   support

            NORMAL       0.93      0.79      0.85       234
PNEUMONIA_bacteria       0.63      0.97      0.76       242
   PNEUMONIA_virus       0.77      0.28      0.41       148

          accuracy                           0.74       624
         macro avg       0.78      0.68      0.67       624
      weighted avg       0.78      0.74      0.71       624

Confusion matrix:
 [[184  38  12]
 [  7 235   0]
 [  6 101  41]]
